Imports

In [1]:
import os
import pandas as pd
import numpy as np
import warnings
warnings.filterwarnings('ignore')

Prep Input and Output Parameters

In [2]:
## Paths Input Here
base_dir = r'/Users/cu135/Dropbox (Partners HealthCare)/resources/datasets/BIDS_AD_DBS_FORNIX/rawdata'
#----------------------------------------------------------------DO NOT TOUCH----------------------------------------------------------------
analysis = fr"<subject_id>/ses-01/single_subject_atrophy"
out_dir = os.path.join(base_dir, analysis)
print('I will save each file to: \n', out_dir)

I will save each file to: 
 /Users/cu135/Dropbox (Partners HealthCare)/resources/datasets/BIDS_AD_DBS_FORNIX/rawdata/<subject_id>/ses-01/single_subject_atrophy


Extract Atrophy Within Region of Interest

___

## Neuroimaging File Extraction

Here's a breakdown of the inputs:

### 0. Base Directorty (CSF)
- **base_directory**: Absolute path to the base directory containing CSF files.

### 1. Cerebrospinal Fluid (CSF)
- **glob_name_pattern**: File pattern to search for CSF files.

### 2. Grey Matter
- **glob_name_pattern**: File pattern to search for grey matter files.

### 3. White Matter
- **glob_name_pattern**: File pattern to search for white matter files.

---

**Instructions**: Please fill out the `base_directory` and `glob_name_pattern` fields for each segment. This will ensure that the extraction process can locate and identify the appropriate neuroimaging files for further analysis.

---

In [3]:
# Shared Base Directory
base_directory = r'/Users/cu135/Dropbox (Partners HealthCare)/resources/datasets/BIDS_AD_DBS_FORNIX/rawdata'

# Shared Glob Path
shared_glob_path = r'*/*/thresholded_tissue_segment_z_scores'

# Just the name of the tissue in the shared_glob_path directory as it appears in the directory (grey_matter, composite, etc)
tissue_to_import = ['grey_matter', 
                'white_matter', 
                'cerebrospinal_fluid', 
                'composite']

In [4]:
def import_dataframes_by_tissue(base_directory, shared_glob_path, tissue_to_import):
    """
    Imports dataframes based on tissue types from specified directories and glob name patterns.
    
    Parameters:
    - base_directory (str): The base directory where the data resides.
    - shared_glob_path (str): The shared directory path for all tissues.
    - tissue_to_import (list): List of tissues to be imported.
    
    Returns:
    - dict: A dictionary containing dataframes for each specified tissue.
    """
    
    from calvin_utils.file_utils.import_matrices import import_matrices_from_folder

    segments_dict = {}
    for tissue in tissue_to_import:
        glob_path = os.path.join(shared_glob_path, ('*'+tissue+'*'))
        segments_dict[tissue] = glob_path

    dataframes_dict = {}
    nan_handler = {'nan': 0, 'posinf':20, 'neginf':-20}
    for k, v in segments_dict.items():
        dataframes_dict[k] = import_matrices_from_folder(connectivity_path=base_directory, file_pattern=v, convert_nan_to_num=nan_handler)
        print(f'Imported data {k} data with {dataframes_dict[k].shape[0]} voxels and {dataframes_dict[k].shape[1]} patients')
        print(f'Example filename per subject: {dataframes_dict[k].columns[0]}')
        print('\n--------------------------------\n')

    return dataframes_dict


In [17]:
imported_dataframes_by_tissue = import_dataframes_by_tissue(base_directory, shared_glob_path, tissue_to_import)

I will search:  /Users/cu135/Dropbox (Partners HealthCare)/resources/datasets/BIDS_AD_DBS_FORNIX/rawdata/*/*/thresholded_tissue_segment_z_scores/*grey_matter*
Imported data grey_matter data with 902629 voxels and 50 patients
Example filename per subject: sub-130_grey_matter_generated_nifti.nii

--------------------------------

I will search:  /Users/cu135/Dropbox (Partners HealthCare)/resources/datasets/BIDS_AD_DBS_FORNIX/rawdata/*/*/thresholded_tissue_segment_z_scores/*white_matter*
Imported data white_matter data with 902629 voxels and 50 patients
Example filename per subject: sub-130_white_matter_generated_nifti.nii

--------------------------------

I will search:  /Users/cu135/Dropbox (Partners HealthCare)/resources/datasets/BIDS_AD_DBS_FORNIX/rawdata/*/*/thresholded_tissue_segment_z_scores/*cerebrospinal_fluid*
Imported data cerebrospinal_fluid data with 902629 voxels and 50 patients
Example filename per subject: sub-130_cerebrospinal_fluid_generated_nifti.nii

-----------------

**Extract Subject ID From File Names**
Using the example filenames that have been printed above, please define a general string:
1) Preceding the subject ID.
2) Proceeding the subject ID. 

In [18]:
def rename_dataframe_subjects(dataframes_dict, preceding_id, proceeding_id):
    """
    Renames the subjects in the provided dataframes based on the split commands.

    Parameters:
    - dataframes_dict (dict): A dictionary containing dataframes with subjects to be renamed.
    - preceding_id (str): The delimiter for taking the part after the split.
    - proceeding_id (str): The delimiter for taking the part before the split.

    Returns:
    - dict: A dictionary containing dataframes with subjects renamed.
    """
    
    from calvin_utils.file_utils.dataframe_utilities import extract_and_rename_subject_id

    split_command_dict = {preceding_id: 1, proceeding_id: 0}
    
    for k, v in dataframes_dict.items():
        dataframes_dict[k] = extract_and_rename_subject_id(dataframe=dataframes_dict[k], split_command_dict=split_command_dict)
        print('Dataframe: ', k)
        display(dataframes_dict[k])
        print('------------- \n')

    return dataframes_dict


In [19]:
preceding_id = 'sub-'
proceeding_id = '_'

In [20]:
thresholded_atrophy_df_dict = rename_dataframe_subjects(imported_dataframes_by_tissue, preceding_id, proceeding_id)

Dataframe:  grey_matter


,130,137,108,101,106,139,107,138,109,131,...,143,120,118,127,111,129,116,142,145,1201
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
902624,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
902625,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
902626,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
902627,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


------------- 

Dataframe:  white_matter


,130,137,108,101,106,139,107,138,109,131,...,143,120,118,127,111,129,116,142,145,1201
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
902624,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
902625,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
902626,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
902627,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


------------- 

Dataframe:  cerebrospinal_fluid


,130,137,108,101,106,139,107,138,109,131,...,143,120,118,127,111,129,116,142,145,1201
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
902624,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
902625,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
902626,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
902627,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


------------- 

Dataframe:  composite


,130,137,108,101,106,139,107,138,109,131,...,143,120,118,127,111,129,116,142,145,1201
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
902624,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
902625,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
902626,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
902627,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


------------- 



# Extract Region of Interest Atrophy

Import Region of Interest Masks

In [44]:
folder_to_import_from = '/Users/cu135/Dropbox (Partners HealthCare)/resources/published_networks/memory_network_thresholded'
shared_glob_pattern = '*.nii'

In [45]:
from calvin_utils.file_utils.import_matrices import import_matrices_from_folder
region_of_interest_df = import_matrices_from_folder(connectivity_path=folder_to_import_from, file_pattern=shared_glob_pattern)
region_of_interest_df

I will search:  /Users/cu135/Dropbox (Partners HealthCare)/resources/published_networks/memory_network_thresholded/*.nii


,between_5th_and_95th_mask.nii,mem_net_poles.nii,whole_brain_mask.nii,below_5_percentile_mask.nii,above_95_percentile_mask.nii
0,0.0,0.0,0.0,0.0,0.0
1,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...
902624,0.0,0.0,0.0,0.0,0.0
902625,0.0,0.0,0.0,0.0,0.0
902626,0.0,0.0,0.0,0.0,0.0
902627,0.0,0.0,0.0,0.0,0.0


Extract Damage Scores Per Region of Interest

In [46]:
def calculate_damage_scores(thresholded_atrophy_df_dict, region_of_interest_df):
    """
    Calculates damage scores for each region of interest based on thresholded atrophy data.
    
    Parameters:
    - thresholded_atrophy_df_dict (dict): Dictionary containing dataframes with thresholded atrophy data.
    - region_of_interest_df (DataFrame): DataFrame containing regions of interest.
    
    Returns:
    - dict: A dictionary containing damage scores for each region of interest for each patient.
    """
    
    damage_df_dict = {}
    
    for k, v in thresholded_atrophy_df_dict.items():
        
        # Initialize the Dataframe
        damage_df_dict[k] = pd.DataFrame(index=v.columns, columns=region_of_interest_df.columns)
        
        # Iterate Over Each Region of Interest
        for matrix in damage_df_dict[k].columns:
            # Extract Damage Score for Each Patient
            for subject in damage_df_dict[k].index:
                # Mask the subject dataframe to the matrix at hand
                intersection = v[subject].where(region_of_interest_df[matrix] > 0, 0)
                
                # Weight the overlapping components by multiplication
                weighted_overlap = intersection * region_of_interest_df[matrix]
                
                # Assess overall damage score
                damage_score = weighted_overlap.sum()
                damage_df_dict[k].loc[subject, matrix] = damage_score
                
        print('Dataframe: ', k)
        display(damage_df_dict[k])
        print('------------- \n')
    
    return damage_df_dict


In [47]:
damage_df_dict = calculate_damage_scores(thresholded_atrophy_df_dict, region_of_interest_df)


Dataframe:  grey_matter


,between_5th_and_95th_mask.nii,mem_net_poles.nii,whole_brain_mask.nii,below_5_percentile_mask.nii,above_95_percentile_mask.nii
130,-8435.009255,-10531.981891,-18966.991146,-4461.941537,-6070.040355
137,-40325.436938,-38968.181558,-79293.618496,-17808.356903,-21159.824655
108,-16535.004377,-13595.10901,-30130.113387,-5364.019621,-8231.089389
101,-12828.914344,-11608.543693,-24437.458037,-4356.66914,-7251.874553
106,-6736.082946,-5630.19479,-12366.277736,-1984.577166,-3645.617624
139,-21680.770137,-19096.08201,-40776.852147,-8477.42463,-10618.65738
107,-21742.934372,-17228.951314,-38971.885687,-5267.319149,-11961.632166
138,-35768.789942,-25944.151733,-61712.941676,-11284.75569,-14659.396044
109,-15292.022505,-12059.575799,-27351.598304,-4303.103673,-7756.472126
131,-7700.35993,-9985.709377,-17686.069307,-2997.542037,-6988.16734


------------- 

Dataframe:  white_matter


,between_5th_and_95th_mask.nii,mem_net_poles.nii,whole_brain_mask.nii,below_5_percentile_mask.nii,above_95_percentile_mask.nii
130,-1383.02269,-578.698774,-1961.721464,-156.355983,-422.342792
137,-3459.477359,-2109.49847,-5568.97583,-152.249295,-1957.249175
108,-10072.310433,-3135.85708,-13208.167513,-816.723372,-2319.133708
101,-2235.846971,-776.516816,-3012.363786,-281.532614,-494.984201
106,-3629.862909,-2681.140404,-6311.003313,-487.330665,-2193.809739
139,-1561.588704,-661.475222,-2223.063926,-232.470018,-429.005204
107,-5534.458933,-2699.780628,-8234.239561,-639.83151,-2059.949119
138,-4851.532683,-1927.527089,-6779.059772,-274.231148,-1653.295941
109,-5258.478326,-1784.330516,-7042.808842,-610.255782,-1174.074734
131,-2132.145479,-1057.598366,-3189.743845,-269.857087,-787.741278


------------- 

Dataframe:  cerebrospinal_fluid


,between_5th_and_95th_mask.nii,mem_net_poles.nii,whole_brain_mask.nii,below_5_percentile_mask.nii,above_95_percentile_mask.nii
130,28728.966401,22080.404832,50809.371233,8680.112006,13400.292826
137,8617.855779,7385.834861,16003.69064,1584.64866,5801.186201
108,36872.174485,26064.012293,62936.186778,11657.958728,14406.053565
101,31469.33124,30255.50435,61724.83559,11267.795434,18987.708916
106,6544.341025,4965.387201,11509.728226,2054.226543,2911.160658
139,24582.856783,18826.057618,43408.914401,10122.012996,8704.044622
107,14778.511616,10019.976005,24798.487621,4876.064955,5143.91105
138,9001.156696,5442.256645,14443.413341,2704.657729,2737.598915
109,52043.290159,39458.023207,91501.313366,20258.840868,19199.182339
131,33260.061784,23128.274478,56388.336261,11349.971547,11778.302931


------------- 

Dataframe:  composite


,between_5th_and_95th_mask.nii,mem_net_poles.nii,whole_brain_mask.nii,below_5_percentile_mask.nii,above_95_percentile_mask.nii
130,38462.191335,33097.746034,71559.937369,13291.508581,19806.237453
137,52331.07984,48324.60247,100655.682309,19542.732886,28781.869583
108,62819.57294,42541.917101,105361.490041,17768.156238,24773.760863
101,46462.444805,42592.76398,89055.208785,15898.758347,26694.005633
106,16881.335754,13271.577537,30152.913291,4523.712637,8747.8649
139,47665.251737,38469.697111,86134.948848,18759.922251,19709.77486
107,42012.975298,29904.293106,71917.268404,10764.574963,19139.718143
138,49553.816678,33275.35944,82829.176118,14258.797857,19016.561583
109,72516.883388,53226.950051,125743.833439,25154.839688,28072.110364
131,42963.691265,33950.833556,76914.524821,14602.303807,19348.529748


------------- 



Extract Total Atrophy Volume

In [48]:
def calculate_total_atrophy_voxels(thresholded_atrophy_df_dict, damage_df_dict):
    """
    Calculates the total number of atrophy voxels within a mask and updates the damage_df_dict with this information.
    
    Parameters:
    - thresholded_atrophy_df_dict (dict): Dictionary containing dataframes with thresholded atrophy data.
    - damage_df_dict (dict): Dictionary containing dataframes to which the 'Total Atrophy Voxels' column will be added.
    
    Returns:
    - dict: The updated damage_df_dict with a new column 'Total Atrophy Voxels'.
    """
    
    import numpy as np
    from nimlab import datasets as nimds
    
    # Get the mask and brain indices
    mni_mask = nimds.get_img("mni_icbm152")
    mask_data = mni_mask.get_fdata().flatten()
    brain_indices = np.where(mask_data > 0)[0]
    
    for k, v in thresholded_atrophy_df_dict.items():
        # initialize the column
        damage_df_dict[k]['Total Atrophy Voxels'] = np.nan
        for patient_id in v.columns:
            # Filter the dataframe using brain indices
            filtered_df = v[patient_id].iloc[brain_indices]
            damage_df_dict[k].loc[patient_id, 'Total Atrophy Voxels'] = np.count_nonzero(filtered_df)
        
        print('Dataframe: ', k)
        display(damage_df_dict[k])
        print('------------- \n')
    
    return damage_df_dict


In [49]:
damage_df_dict = calculate_total_atrophy_voxels(thresholded_atrophy_df_dict, damage_df_dict)

Dataframe:  grey_matter


,between_5th_and_95th_mask.nii,mem_net_poles.nii,whole_brain_mask.nii,below_5_percentile_mask.nii,above_95_percentile_mask.nii,Total Atrophy Voxels
130,-8435.009255,-10531.981891,-18966.991146,-4461.941537,-6070.040355,6992.0
137,-40325.436938,-38968.181558,-79293.618496,-17808.356903,-21159.824655,26781.0
108,-16535.004377,-13595.10901,-30130.113387,-5364.019621,-8231.089389,10519.0
101,-12828.914344,-11608.543693,-24437.458037,-4356.66914,-7251.874553,8731.0
106,-6736.082946,-5630.19479,-12366.277736,-1984.577166,-3645.617624,4426.0
139,-21680.770137,-19096.08201,-40776.852147,-8477.42463,-10618.65738,13139.0
107,-21742.934372,-17228.951314,-38971.885687,-5267.319149,-11961.632166,13976.0
138,-35768.789942,-25944.151733,-61712.941676,-11284.75569,-14659.396044,20272.0
109,-15292.022505,-12059.575799,-27351.598304,-4303.103673,-7756.472126,9663.0
131,-7700.35993,-9985.709377,-17686.069307,-2997.542037,-6988.16734,6448.0


------------- 

Dataframe:  white_matter


,between_5th_and_95th_mask.nii,mem_net_poles.nii,whole_brain_mask.nii,below_5_percentile_mask.nii,above_95_percentile_mask.nii,Total Atrophy Voxels
130,-1383.02269,-578.698774,-1961.721464,-156.355983,-422.342792,766.0
137,-3459.477359,-2109.49847,-5568.97583,-152.249295,-1957.249175,2147.0
108,-10072.310433,-3135.85708,-13208.167513,-816.723372,-2319.133708,4757.0
101,-2235.846971,-776.516816,-3012.363786,-281.532614,-494.984201,1120.0
106,-3629.862909,-2681.140404,-6311.003313,-487.330665,-2193.809739,2343.0
139,-1561.588704,-661.475222,-2223.063926,-232.470018,-429.005204,779.0
107,-5534.458933,-2699.780628,-8234.239561,-639.83151,-2059.949119,3050.0
138,-4851.532683,-1927.527089,-6779.059772,-274.231148,-1653.295941,2549.0
109,-5258.478326,-1784.330516,-7042.808842,-610.255782,-1174.074734,2654.0
131,-2132.145479,-1057.598366,-3189.743845,-269.857087,-787.741278,1263.0


------------- 

Dataframe:  cerebrospinal_fluid


,between_5th_and_95th_mask.nii,mem_net_poles.nii,whole_brain_mask.nii,below_5_percentile_mask.nii,above_95_percentile_mask.nii,Total Atrophy Voxels
130,28728.966401,22080.404832,50809.371233,8680.112006,13400.292826,12566.0
137,8617.855779,7385.834861,16003.69064,1584.64866,5801.186201,3627.0
108,36872.174485,26064.012293,62936.186778,11657.958728,14406.053565,13124.0
101,31469.33124,30255.50435,61724.83559,11267.795434,18987.708916,15272.0
106,6544.341025,4965.387201,11509.728226,2054.226543,2911.160658,3032.0
139,24582.856783,18826.057618,43408.914401,10122.012996,8704.044622,10233.0
107,14778.511616,10019.976005,24798.487621,4876.064955,5143.91105,6087.0
138,9001.156696,5442.256645,14443.413341,2704.657729,2737.598915,3142.0
109,52043.290159,39458.023207,91501.313366,20258.840868,19199.182339,21124.0
131,33260.061784,23128.274478,56388.336261,11349.971547,11778.302931,12997.0


------------- 

Dataframe:  composite


,between_5th_and_95th_mask.nii,mem_net_poles.nii,whole_brain_mask.nii,below_5_percentile_mask.nii,above_95_percentile_mask.nii,Total Atrophy Voxels
130,38462.191335,33097.746034,71559.937369,13291.508581,19806.237453,17240.0
137,52331.07984,48324.60247,100655.682309,19542.732886,28781.869583,30920.0
108,62819.57294,42541.917101,105361.490041,17768.156238,24773.760863,23894.0
101,46462.444805,42592.76398,89055.208785,15898.758347,26694.005633,20849.0
106,16881.335754,13271.577537,30152.913291,4523.712637,8747.8649,8922.0
139,47665.251737,38469.697111,86134.948848,18759.922251,19709.77486,21691.0
107,42012.975298,29904.293106,71917.268404,10764.574963,19139.718143,20189.0
138,49553.816678,33275.35944,82829.176118,14258.797857,19016.561583,24304.0
109,72516.883388,53226.950051,125743.833439,25154.839688,28072.110364,28436.0
131,42963.691265,33950.833556,76914.524821,14602.303807,19348.529748,17929.0


------------- 



Organize the Subjects

In [56]:
def sort_dataframes_by_index(damage_df_dict):
    """
    Attempts to convert the index of each dataframe in damage_df_dict to integers 
    and then sorts the dataframe by its index in ascending order.
    
    Parameters:
    - damage_df_dict (dict): Dictionary containing dataframes to be sorted.
    
    Returns:
    - dict: The sorted damage_df_dict.
    """
    
    sorted_df_dict = {}
    
    for k, df in damage_df_dict.items():
        try:
            # Convert index to integers
            df.index = df.index.astype(int)
            
            # Sort dataframe by index
            sorted_df = df.sort_index(ascending=True)
            sorted_df_dict[k] = sorted_df
            
            # Display the results
            print('Dataframe: ', k)
            display(sorted_df)
            print('------------- \n')
            
        except ValueError:
            # If conversion to integer fails, just use the original dataframe
            sorted_df_dict[k] = df

    return sorted_df_dict


In [57]:
sorted_damage_df_dict = sort_dataframes_by_index(damage_df_dict)

Dataframe:  grey_matter


,between_5th_and_95th_mask.nii,mem_net_poles.nii,whole_brain_mask.nii,below_5_percentile_mask.nii,above_95_percentile_mask.nii,Total Atrophy Voxels
101,-12828.914344,-11608.543693,-24437.458037,-4356.66914,-7251.874553,8731.0
102,-20235.500834,-16292.256178,-36527.757012,-6044.287386,-10247.968792,12827.0
103,-15050.237761,-14387.012007,-29437.249768,-5891.712589,-8495.299418,10463.0
104,-17709.509254,-14207.099872,-31916.609126,-6565.680288,-7641.419584,11277.0
105,-11986.879694,-9757.281379,-21744.161073,-5374.584977,-4382.696402,7913.0
106,-6736.082946,-5630.19479,-12366.277736,-1984.577166,-3645.617624,4426.0
107,-21742.934372,-17228.951314,-38971.885687,-5267.319149,-11961.632166,13976.0
108,-16535.004377,-13595.10901,-30130.113387,-5364.019621,-8231.089389,10519.0
109,-15292.022505,-12059.575799,-27351.598304,-4303.103673,-7756.472126,9663.0
110,-17680.422868,-13030.775222,-30711.19809,-6377.560962,-6653.21426,10973.0


------------- 

Dataframe:  white_matter


,between_5th_and_95th_mask.nii,mem_net_poles.nii,whole_brain_mask.nii,below_5_percentile_mask.nii,above_95_percentile_mask.nii,Total Atrophy Voxels
101,-2235.846971,-776.516816,-3012.363786,-281.532614,-494.984201,1120.0
102,-15190.662228,-6185.89652,-21376.558749,-970.271303,-5215.625218,8175.0
103,-5616.442869,-3560.75068,-9177.193549,-880.926793,-2679.823887,3439.0
104,-3605.923696,-1958.289535,-5564.213231,-575.844155,-1382.44538,2147.0
105,-3127.452919,-1482.601797,-4610.054716,-407.690999,-1074.910798,1770.0
106,-3629.862909,-2681.140404,-6311.003313,-487.330665,-2193.809739,2343.0
107,-5534.458933,-2699.780628,-8234.239561,-639.83151,-2059.949119,3050.0
108,-10072.310433,-3135.85708,-13208.167513,-816.723372,-2319.133708,4757.0
109,-5258.478326,-1784.330516,-7042.808842,-610.255782,-1174.074734,2654.0
110,-4467.04747,-1656.719862,-6123.767332,-440.217388,-1216.502475,2262.0


------------- 

Dataframe:  cerebrospinal_fluid


,between_5th_and_95th_mask.nii,mem_net_poles.nii,whole_brain_mask.nii,below_5_percentile_mask.nii,above_95_percentile_mask.nii,Total Atrophy Voxels
101,31469.33124,30255.50435,61724.83559,11267.795434,18987.708916,15272.0
102,15702.729942,13153.907252,28856.637194,5802.28787,7351.619382,7029.0
103,31310.721282,25287.844167,56598.565449,10808.674522,14479.169644,12868.0
104,28279.478548,19059.103446,47338.581993,8923.007053,10136.096393,10426.0
105,7834.297598,5216.731562,13051.02916,3548.859316,1667.872246,3149.0
106,6544.341025,4965.387201,11509.728226,2054.226543,2911.160658,3032.0
107,14778.511616,10019.976005,24798.487621,4876.064955,5143.91105,6087.0
108,36872.174485,26064.012293,62936.186778,11657.958728,14406.053565,13124.0
109,52043.290159,39458.023207,91501.313366,20258.840868,19199.182339,21124.0
110,54077.518145,34719.130097,88796.648242,16464.893046,18254.237051,18654.0


------------- 

Dataframe:  composite


,between_5th_and_95th_mask.nii,mem_net_poles.nii,whole_brain_mask.nii,below_5_percentile_mask.nii,above_95_percentile_mask.nii,Total Atrophy Voxels
101,46462.444805,42592.76398,89055.208785,15898.758347,26694.005633,20849.0
102,50998.546344,35440.825184,86439.371528,12756.365907,22684.459277,24230.0
103,51788.585485,43095.163088,94883.748573,17539.121777,25556.041311,22327.0
104,49434.306385,35009.799461,84444.105846,15978.659719,19031.139742,19756.0
105,22896.923907,16428.523467,39325.447374,9313.295481,7115.227985,11670.0
106,16881.335754,13271.577537,30152.913291,4523.712637,8747.8649,8922.0
107,42012.975298,29904.293106,71917.268404,10764.574963,19139.718143,20189.0
108,62819.57294,42541.917101,105361.490041,17768.156238,24773.760863,23894.0
109,72516.883388,53226.950051,125743.833439,25154.839688,28072.110364,28436.0
110,76082.633489,49267.102891,125349.73638,23251.007801,26016.09509,26745.0


------------- 



Save the Results

In [58]:
import os
from calvin_utils.nifti_utils.generate_nifti import view_and_save_nifti
from tqdm import tqdm

def save_csv_to_bids(dataframes_dict, bids_base_dir, analysis='atrophy_results', ses=None, dry_run=True):
    """
    Saves csv to a BIDS directory structure.
    
    Parameters:
    - dataframes_dict (dict): Dictionary containing dataframes with NIFTI data.
    - bids_base_dir (str): The base directory where the BIDS structure starts.
    - ses (str, optional): Session identifier. If None, defaults to '01'.
    
    Note:
    This function assumes a predefined BIDS directory structure and saves the CSV 
    accordingly.
    """
    
    for key, value in tqdm(dataframes_dict.items()):            
        # Define BIDS Directory Architecture
        sub_no = 'all'
        if ses is None:
            ses_no = '01'
        else:
            ses_no = ses
        
        # Define and Initialize the Save Directory
        out_dir = os.path.join(bids_base_dir, 'neuroimaging_analyses', f'ses-{ses_no}', f'sub-{sub_no}', analysis)
        os.makedirs(out_dir, exist_ok=True)
        
        # Save Image to BIDS Directory
        if dry_run:
            print(out_dir+f'/{key}.csv')
        else:
            value.to_csv(out_dir+f'/{key}.csv')
            print('Saved to: ', out_dir+f'/{key}.csv')

In [59]:
bids_base_dir = '/Users/cu135/Dropbox (Partners HealthCare)/resources/datasets/BIDS_AD_DBS_FORNIX'
analysis='memory_net_atrophy_results'

In [60]:
save_csv_to_bids(sorted_damage_df_dict, bids_base_dir, analysis, ses=None, dry_run=False)

100%|██████████| 4/4 [00:00<00:00, 630.94it/s]

Saved to:  /Users/cu135/Dropbox (Partners HealthCare)/resources/datasets/BIDS_AD_DBS_FORNIX/neuroimaging_analyses/ses-01/sub-all/memory_net_atrophy_results/grey_matter.csv
Saved to:  /Users/cu135/Dropbox (Partners HealthCare)/resources/datasets/BIDS_AD_DBS_FORNIX/neuroimaging_analyses/ses-01/sub-all/memory_net_atrophy_results/white_matter.csv
Saved to:  /Users/cu135/Dropbox (Partners HealthCare)/resources/datasets/BIDS_AD_DBS_FORNIX/neuroimaging_analyses/ses-01/sub-all/memory_net_atrophy_results/cerebrospinal_fluid.csv
Saved to:  /Users/cu135/Dropbox (Partners HealthCare)/resources/datasets/BIDS_AD_DBS_FORNIX/neuroimaging_analyses/ses-01/sub-all/memory_net_atrophy_results/composite.csv


All done. Enjoy your analyses. 

--Calvin 